Uber is a taxi service provider as we know, we need to predict the high
booking area using an Unsupervised algorithm and price for the location using a
supervised algorithm and use some map function to display the data

In [1]:
import folium
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("C:\\Users\\MGC\\Downloads\\assignment\\rideshare_kaggle.csv\\rideshare_kaggle.csv")

In [3]:
df = df.sample(50000)

In [4]:
df.isna().sum()

id                                0
timestamp                         0
hour                              0
day                               0
month                             0
datetime                          0
timezone                          0
source                            0
destination                       0
cab_type                          0
product_id                        0
name                              0
price                          3936
distance                          0
surge_multiplier                  0
latitude                          0
longitude                         0
temperature                       0
apparentTemperature               0
short_summary                     0
long_summary                      0
precipIntensity                   0
precipProbability                 0
humidity                          0
windSpeed                         0
windGust                          0
windGustTime                      0
visibility                  

In [5]:
df["price"] = df["price"].fillna(value=np.mean(df["price"]))

In [6]:
# Select relevant columns for clustering
cluster_data = df[['latitude', 'longitude']]

# Perform K-means clustering
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(cluster_data)

# Add cluster labels to the DataFrame
df['cluster_label'] = kmeans.labels_


In [7]:
# Select relevant features for regression
regression_data = df[['distance', 'cab_type', 'surge_multiplier', 'temperature', 'price']]

# Convert categorical variables into dummy variables
regression_data = pd.get_dummies(regression_data)

# Split the data into training and testing sets
X = regression_data.drop('price', axis=1)
y = regression_data['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a linear regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

new_locations_encoded = pd.get_dummies(X_test)
predicted_prices = reg_model.predict(new_locations_encoded)


In [22]:
from sklearn.metrics import mean_squared_error

# Assuming you have the true prices from the dataset
true_prices = y_test

# Compute the mean squared error
mse = mean_squared_error(true_prices, predicted_prices)

print("Mean Squared Error:", mse)


Mean Squared Error: 67.4451454229723


In [19]:
import random
# Define the number of samples to display on the map
num_samples = 1000  # Adjust this number as per your preference and system capabilities

# Randomly select a subset of samples
random_indices = random.sample(range(len(df)), min(num_samples, len(df)))
sampled_df = df.iloc[random_indices]

# Create a map centered on a specific location
map_center = [42.3647, -71.0542]  # Latitude and longitude of the map center
map_zoom = 12  # Zoom level of the map

# Create the map object
m = folium.Map(location=map_center, zoom_start=map_zoom)

# Add markers for each sampled Uber trip
for _, row in sampled_df.iterrows():
    lat = row['latitude']
    lon = row['longitude']
    
    # Add a marker for each Uber trip
    folium.Marker([lat, lon]).add_to(m)

# Display the map
m
